# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,475.761421,-0.360854,1234,475.400568,476.122275,950.801136,952.244550
6,445.318174,0.913487,1234,446.231662,444.404687,892.463323,888.809374
7,785.059971,-0.459766,1234,784.600205,785.519737,1569.200411,1571.039475
8,42.809692,0.133308,1234,42.943000,42.676384,85.886001,85.352769
9,963.461213,0.374370,1234,963.835583,963.086843,1927.671167,1926.173686
10,565.102833,-0.570201,1234,564.532633,565.673034,1129.065265,1131.346068
11,465.095948,0.340611,1234,465.436559,464.755338,930.873118,929.510676
12,363.007852,-0.967412,1234,362.040440,363.975264,724.080880,727.950527
13,624.420935,-0.458697,1234,623.962238,624.879631,1247.924476,1249.759263
14,85.677975,0.077084,1234,85.755058,85.600891,171.510116,171.201782


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-475.400568,476.122275
6,-446.231662,444.404687
7,-784.600205,785.519737
8,-42.943000,42.676384
9,-963.835583,963.086843
10,-564.532633,565.673034
11,-465.436559,464.755338
12,-362.040440,363.975264
13,-623.962238,624.879631
14,-85.755058,85.600891


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-475.400568,476.122275
6,-446.231662,444.404687
7,-784.600205,785.519737
8,-42.943000,42.676384
9,-963.835583,963.086843
10,-564.532633,565.673034
11,-465.436559,464.755338
12,-362.040440,363.975264
13,-623.962238,624.879631
14,-85.755058,85.600891


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-475.40056779,  476.12227485],
       [-446.23166166,  444.40468693],
       [-784.60020548,  785.51973749],
       [ -42.94300047,   42.67638436],
       [-963.83558333,  963.08684287],
       [-564.53263264,  565.673034  ],
       [-465.436559  ,  464.75533787],
       [-362.04044009,  363.97526358],
       [-623.96223806,  624.87963141],
       [ -85.75505822,   85.60089114]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,948.801136,-952.244550
6,890.463323,-888.809374
7,1567.200411,-1571.039475
8,83.886001,-85.352769
9,1925.671167,-1926.173686
10,1127.065265,-1131.346068
11,928.873118,-929.510676
12,722.080880,-727.950527
13,1245.924476,-1249.759263
14,169.510116,-171.201782


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.475761,0.319573
6,0.445318,0.956744
7,0.785060,0.270117
8,0.042810,0.566654
9,0.963461,0.687185
10,0.565103,0.214900
11,0.465096,0.670305
12,0.363008,0.016294
13,0.624421,0.270652
14,0.085678,0.538542


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,475.761421,-0.360854
6,445.318174,0.913487
7,785.059971,-0.459766
8,42.809692,0.133308
9,963.461213,0.374370
10,565.102833,-0.570201
11,465.095948,0.340611
12,363.007852,-0.967412
13,624.420935,-0.458697
14,85.677975,0.077084


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
